# Clinvar exploration notebook



In [8]:
import os
import polars as pl
import polars_bio as pb
from pathlib import Path
from pycomfort import files
import polars_bio as pb
# Configure Polars to show more rows and columns
pl.Config.set_tbl_rows(-1)  # Show all rows (or use a large number like 1000)
pl.Config.set_tbl_cols(-1)  # Show all columns (or use a large number like 50)
pl.Config.set_tbl_width_chars(1000)  # Increase table width to prevent column truncation
pl.Config.set_fmt_str_lengths(1000)  # Show longer string values without truncation







polars.config.Config

In [9]:
from pycomfort.logging import to_nice_stdout, to_nice_file
to_nice_stdout()

Ensembl test
============

In [10]:
from pycomfort import files
variations = Path("/home/antonkulaga/.cache/ensembl_variation")
files.tprint(variations)

ensembl_variation
	homo_sapiens-chr1.vcf.gz.csi
	homo_sapiens-chr1.vcf.gz
	homo_sapiens-chr21.parquet
	homo_sapiens-chr21.vcf.gz
	homo_sapiens-chr22.vcf.gz.csi
	homo_sapiens-chr22.parquet
	homo_sapiens-chr1.parquet
	homo_sapiens-chr22.vcf.gz
	homo_sapiens-chr1.vcf
	test
		homo_sapiens-chr1-cleaned.vcf
		homo_sapiens-chr21-cleaned.vcf
		homo_sapiens-chr21-cleaned.parquet
		homo_sapiens-chr21.vcf
		homo_sapiens-chr1.parquet
		homo_sapiens-chr22.vcf
		homo_sapiens-chr1.vcf


In [11]:
from genobear.downloaders.ensembl_downloader import EnsemblDownloader
downloader = EnsemblDownloader()


e309d972-baf6-4b42-989c-834a28a4862b
└── ensembl_setup/1 ⇒ started 2025-08-16 01:19:38Z
    └── use_checksums: True

e309d972-baf6-4b42-989c-834a28a4862b
└── ensembl_setup/1 ⇒ started 2025-08-16 01:19:38Z
    └── use_checksums: True

e309d972-baf6-4b42-989c-834a28a4862b
└── ensembl_setup/1 ⇒ started 2025-08-16 01:19:38Z
    └── use_checksums: True

e309d972-baf6-4b42-989c-834a28a4862b
└── ensembl_setup/1 ⇒ started 2025-08-16 01:19:38Z
    └── use_checksums: True

e309d972-baf6-4b42-989c-834a28a4862b
└── <unnamed>
    └── discover_chromosomes/2/1 ⇒ started 2025-08-16 01:19:38Z

e309d972-baf6-4b42-989c-834a28a4862b
└── <unnamed>
    └── discover_chromosomes/2/1 ⇒ started 2025-08-16 01:19:38Z

e309d972-baf6-4b42-989c-834a28a4862b
└── <unnamed>
    └── discover_chromosomes/2/1 ⇒ started 2025-08-16 01:19:38Z

e309d972-baf6-4b42-989c-834a28a4862b
└── <unnamed>
    └── discover_chromosomes/2/1 ⇒ started 2025-08-16 01:19:38Z

e309d972-baf6-4b42-989c-834a28a4862b
└── <unnamed>
    └── <unnamed>

In [ ]:
merged_files = downloader.merge()
merged_files

d1c697b4-9b4d-46c9-9dab-7f658b722f07
└── detected f25 parquet files in downloaded cache, splitting and merging them/1 ⇒ started 2025-08-16 01:19:43Z
    ├── explode_snv_alt: True
    ├── parquets: 
    │   ├── 0: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chr2.parquet')
    │   ├── 1: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chr9.parquet')
    │   ├── 2: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chr17.parquet')
    │   ├── 3: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chr21.parquet')
    │   ├── 4: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chr8.parquet')
    │   ├── 5: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chrY.parquet')
    │   ├── 6: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chr20.parquet')
    │   ├── 7: PosixPath('/home/antonkulaga/.cache/ensembl_variations/homo_sapiens-chr19.parquet')
    │   ├── 8:

In [ ]:
from genobear.downloaders.uploader import HuggingFaceUploader
from dotenv import load_dotenv
if (Path.cwd / ".env").exists():
    load_dotenv()
else:
    load_dotenv(Path.cwd.parent)
uploader = HuggingFaceUploader(hf_repo="just-dna-seq/ensembl_variations", hf_token=os.getenv("HF_TOKEN"))
uploader.upload_folder(Path("/home/antonkulaga/.cache/ensembl_variations/merged"), path_in_repo="ensembl_variations")

Clinvar descriptions
====================

In [ ]:
clinvar_folder= genobear_folder / "databases" / "clinvar"
clinvar_vcf_gz = clinvar_folder / "hg38" / "clinvar.vcf.gz"
clinvar_vcf = clinvar_folder / "hg38" / "clinvar.vcf"

assert clinvar_vcf.exists(), f"Clinvar VCF file {clinvar_vcf} does not exist"

clinvar_parquet = clinvar_folder / "clinvar_hg38.parquet"

In [ ]:
pb.describe_vcf(str(clinvar_vcf))

name,type,description
str,str,str
"""AF_ESP""","""Float""","""allele frequencies from GO-ESP"""
"""AF_EXAC""","""Float""","""allele frequencies from ExAC"""
"""AF_TGP""","""Float""","""allele frequencies from TGP"""
"""ALLELEID""","""Integer""","""the ClinVar Allele ID"""
"""CLNDN""","""String""","""ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDNINCL""","""String""","""For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDISDB""","""String""","""Tag-value pairs of disease database name and identifier submitted for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNDISDBINCL""","""String""","""For included Variant: Tag-value pairs of disease database name and identifier for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNHGVS""","""String""","""Top-level (primary assembly, alt, or patch) HGVS expression."""


In [ ]:
gb.get_info_fields(str(clinvar_vcf))

AttributeError: module 'genobear' has no attribute 'get_info_fields'

In [ ]:
gb.get_info_fields(str(clinvar_vcf))

AttributeError: module 'genobear' has no attribute 'get_info_fields'

In [ ]:
gb.get_info_fields(str(clinvar_vcf))

AttributeError: module 'genobear' has no attribute 'get_info_fields'

In [ ]:
from genobear.io import read_vcf_file
vcf = clinvar_folder / "hg38" / "clinvar.vcf"
clinvar = read_vcf_file(vcf, to_parquet=False, streaming=False)
clinvar.head().collect()

chrom,start,end,id,ref,alt,qual,filter,af_esp,af_exac,af_tgp,alleleid,clndn,clndnincl,clndisdb,clndisdbincl,clnhgvs,clnrevstat,clnsig,clnsigconf,clnsigincl,clnsigscv,clnvc,clnvcso,clnvi,dbvarid,geneinfo,mc,oncdn,oncdnincl,oncdisdb,oncdisdbincl,onc,oncincl,oncrevstat,oncscv,oncconf,origin,rs,scidn,scidnincl,scidisdb,scidisdbincl,scirevstat,sci,sciincl,sciscv
str,u32,u32,str,str,str,f64,str,f32,f32,f32,i32,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str]
"""1""",66926,66927,"""3385321""","""AG""","""A""",0.0,"""""",null,null,null,3544463,"[""Retinitis_pigmentosa""]",null,"[""Human_Phenotype_Ontology:HP:0000547"", ""MONDO:MONDO:0019200"", … ""Orphanet:791""]",null,"[""NC_000001.11:g.66927del""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV005419006""]","""Deletion""","""SO:0000159""",null,null,"""OR4F5:79501""","[""SO:0001627|intron_variant""]",null,null,null,null,null,null,null,null,null,"[""0""]",null,null,null,null,null,null,null,null,null
"""1""",69134,69134,"""2205837""","""A""","""G""",0.0,"""""",null,null,null,2193183,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69134A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Likely_benign""]",null,null,"[""SCV003526545""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA502008""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""781394307""]",null,null,null,null,null,null,null,null
"""1""",69308,69308,"""3925305""","""A""","""G""",0.0,"""""",null,null,null,4039319,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69308A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120601""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null
"""1""",69314,69314,"""3205580""","""T""","""G""",0.0,"""""",null,null,null,3374047,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69314T>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV004995495""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA338197388""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""2521653848""]",null,null,null,null,null,null,null,null
"""1""",69404,69404,"""3925306""","""T""","""C""",0.0,"""""",null,null,null,4039320,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69404T>C""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120602""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null


Load Genomic Sample
===================

In [ ]:
from genobear.io import read_vcf_file
from pathlib import Path

data_folder = current_folder / "data"
test_folder = data_folder / "tests"

assert test_folder.exists(), f"Test folder {test_folder} does not exist"


In [ ]:
sample_vcf = test_folder / "antonkulaga.vcf"
sample = read_vcf_file(sample_vcf, info_fields=[], streaming=False)
sample.head().collect()

chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""1""",10009,10009,"""""","""A""","""AC""",0.0,"""RefCall"""
"""1""",10015,10015,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10021,10021,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10027,10027,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10033,10033,"""""","""A""","""G""",0.0,"""RefCall"""


## Annotation with clinvar

In [ ]:
sample.columns

/tmp/ipykernel_178433/374950290.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  sample.columns


['chrom', 'start', 'end', 'id', 'ref', 'alt', 'qual', 'filter']

In [ ]:
sample.join(clinvar, on=["chrom", "start", "end"], how="left").collect()

In [ ]:
pb.describe_vcf(str(clinvar_vcf))

name,type,description
str,str,str
"""AF_ESP""","""Float""","""allele frequencies from GO-ESP"""
"""AF_EXAC""","""Float""","""allele frequencies from ExAC"""
"""AF_TGP""","""Float""","""allele frequencies from TGP"""
"""ALLELEID""","""Integer""","""the ClinVar Allele ID"""
"""CLNDN""","""String""","""ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDNINCL""","""String""","""For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDISDB""","""String""","""Tag-value pairs of disease database name and identifier submitted for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNDISDBINCL""","""String""","""For included Variant: Tag-value pairs of disease database name and identifier for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNHGVS""","""String""","""Top-level (primary assembly, alt, or patch) HGVS expression."""


In [ ]:
# Test the function with both regular and gzipped VCF files
sample_small = read_vcf_file(sample_small_vcf)

NameError: name 'sample_small_vcf' is not defined

In [ ]:
data_folder = Path("../data").absolute().resolve()
test_folder = data_folder / "test"

files.tprint(test_folder)


test
	longevity_snps_1000genom.vcf.gz
	antonkulaga.vcf
	antku_small.vcf
	longevity_snps_1000genom.vcf


## Samples

Loading samples

In [ ]:
sample_antku = pb.read_vcf(str(test_folder / "antku_small.vcf"))
sample_antonkulaga = pb.read_vcf(str(test_folder / "antonkulaga.vcf"))
sample_antku.head().collect()

chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""1""",10009,10009,"""""","""A""","""AC""",0.0,"""RefCall"""
"""1""",10015,10015,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10021,10021,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10027,10027,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10033,10033,"""""","""A""","""G""",0.0,"""RefCall"""


In [ ]:
sample_antku.count().collect()

793rows [00:00, 750300.72rows/s]


chrom,start,end,id,ref,alt,qual,filter
u32,u32,u32,u32,u32,u32,u32,u32
793,793,793,793,793,793,793,793


In [ ]:
sample_antonkulaga.count().collect()

6079744rows [00:04, 1414880.55rows/s]


chrom,start,end,id,ref,alt,qual,filter
u32,u32,u32,u32,u32,u32,u32,u32
6079744,6079744,6079744,6079744,6079744,6079744,6079744,6079744


In [ ]:
# Let's examine the ALT column structure first
# Look at some data to understand the pipe-separated ALT values
sample_data = sample_antku.head(50).collect()
print("Sample data columns:", sample_data.columns)
print("\nFirst few rows:")
print(sample_data.select(["chrom", "pos", "ref", "alt"]).head(10))

# Check for pipe-separated ALT alleles
alt_with_pipes = sample_data.filter(pl.col("alt").str.contains(r"\|")).select("alt")
print(f"\nALT values with pipes: {alt_with_pipes.height}")
if alt_with_pipes.height > 0:
    print("Examples:")
    print(alt_with_pipes.head(5))


In [ ]:
# Polars code to flatmap pipe-separated ALT alleles into separate rows
import polars as pl

def flatmap_alt_alleles(df: pl.LazyFrame, alt_column: str = "alt", separator: str = "|") -> pl.LazyFrame:
    """
    Flatmap pipe-separated ALT alleles into separate rows.
    
    Args:
        df: Input LazyFrame with pipe-separated ALT alleles
        alt_column: Name of the ALT column (default: "alt")
        separator: Separator used in ALT field (default: "|")
    
    Returns:
        LazyFrame with each ALT allele in a separate row
    """
    return (
        df
        .with_columns(
            # Split the ALT column on the separator
            pl.col(alt_column).str.split(separator).alias(f"{alt_column}_split")
        )
        .explode(f"{alt_column}_split")  # Explode to create separate rows
        .drop(alt_column)  # Remove original ALT column
        .rename({f"{alt_column}_split": alt_column})  # Rename back to original column name
        .filter(pl.col(alt_column).str.len() > 0)  # Remove empty strings
    )

# Example usage with sample data
# Let's create a test DataFrame with pipe-separated ALT alleles
test_data = pl.DataFrame({
    "chrom": ["chr1", "chr1", "chr2", "chr3"],
    "pos": [1000, 2000, 3000, 4000],
    "ref": ["A", "G", "T", "C"],
    "alt": ["T|C", "A", "G|A|C", "T|G"],
    "quality": [30, 25, 40, 35]
})

print("Original data:")
print(test_data)
print("\nAfter flatmapping ALT alleles:")
result = flatmap_alt_alleles(test_data.lazy()).collect()
print(result)


In [ ]:
# Alternative approach: Handle both comma and pipe separators
def flatmap_alt_alleles_flexible(df: pl.LazyFrame, alt_column: str = "alt") -> pl.LazyFrame:
    """
    Flatmap ALT alleles with flexible separator detection (handles both , and |).
    
    Args:
        df: Input LazyFrame with separated ALT alleles
        alt_column: Name of the ALT column (default: "alt")
    
    Returns:
        LazyFrame with each ALT allele in a separate row
    """
    return (
        df
        .with_columns(
            # First replace pipes with commas, then split on commas
            pl.col(alt_column)
            .str.replace_all(r"\|", ",")
            .str.split(",")
            .alias(f"{alt_column}_split")
        )
        .explode(f"{alt_column}_split")
        .drop(alt_column)
        .rename({f"{alt_column}_split": alt_column})
        .filter(pl.col(alt_column).str.len() > 0)  # Remove empty strings
        .with_columns(
            pl.col(alt_column).str.strip_chars()  # Remove any whitespace
        )
    )

# Test with mixed separators
mixed_data = pl.DataFrame({
    "chrom": ["chr1", "chr1", "chr2", "chr3", "chr4"],
    "pos": [1000, 2000, 3000, 4000, 5000],
    "ref": ["A", "G", "T", "C", "A"],
    "alt": ["T|C", "A,G", "G|A|C", "T,G", "C"],
    "quality": [30, 25, 40, 35, 45]
})

print("Mixed separator data:")
print(mixed_data)
print("\nAfter flexible flatmapping:")
flexible_result = flatmap_alt_alleles_flexible(mixed_data.lazy()).collect()
print(flexible_result)


In [ ]:
# Advanced version: Preserve original row info and add allele index
def flatmap_alt_alleles_with_index(df: pl.LazyFrame, alt_column: str = "alt", separator: str = "|") -> pl.LazyFrame:
    """
    Flatmap ALT alleles while preserving original row information and adding allele index.
    
    Args:
        df: Input LazyFrame with separated ALT alleles
        alt_column: Name of the ALT column (default: "alt")
        separator: Separator used in ALT field (default: "|")
    
    Returns:
        LazyFrame with each ALT allele in a separate row, including original_row_id and allele_index
    """
    return (
        df
        .with_row_index("original_row_id")  # Add original row ID
        .with_columns(
            pl.col(alt_column).str.split(separator).alias(f"{alt_column}_split")
        )
        .explode(f"{alt_column}_split")
        .with_columns(
            # Add allele index within each original row
            pl.int_range(pl.len()).over("original_row_id").alias("allele_index")
        )
        .drop(alt_column)
        .rename({f"{alt_column}_split": alt_column})
        .filter(pl.col(alt_column).str.len() > 0)
    )

# Test with the advanced version
print("Advanced version with row ID and allele index:")
advanced_result = flatmap_alt_alleles_with_index(test_data.lazy()).collect()
print(advanced_result)

# Show how many alleles each original row had
allele_counts = (
    advanced_result
    .group_by("original_row_id")
    .agg(
        pl.col("allele_index").max().alias("max_allele_index"),
        pl.col("alt").count().alias("allele_count"),
        pl.col("alt").alias("alleles")
    )
    .sort("original_row_id")
)
print("\nAllele counts per original row:")
print(allele_counts)


In [ ]:
# Practical example: Apply to your actual VCF data
def process_vcf_with_alt_splitting(vcf_lazy_frame: pl.LazyFrame) -> pl.LazyFrame:
    """
    Process a VCF LazyFrame to split pipe-separated ALT alleles.
    Designed to work with genobear VCF reading functions.
    
    Args:
        vcf_lazy_frame: LazyFrame from VCF file (e.g., from gb.read_vcf_file)
    
    Returns:
        LazyFrame with exploded ALT alleles
    """
    # Check if the DataFrame has the expected VCF columns
    schema = vcf_lazy_frame.collect_schema()
    alt_col = None
    
    # Find the ALT column (could be 'alt', 'ALT', etc.)
    for col_name in schema.names():
        if col_name.lower() == 'alt':
            alt_col = col_name
            break
    
    if alt_col is None:
        raise ValueError("No ALT column found in the VCF data")
    
    return flatmap_alt_alleles_flexible(vcf_lazy_frame, alt_column=alt_col)

# Example of how to use with your actual data:
print("Usage examples:")
print("1. Basic flatmapping:")
print("   result = flatmap_alt_alleles(your_vcf_lazy_frame)")
print()
print("2. Flexible separator handling (comma and pipe):")
print("   result = flatmap_alt_alleles_flexible(your_vcf_lazy_frame)")
print()
print("3. With row tracking:")
print("   result = flatmap_alt_alleles_with_index(your_vcf_lazy_frame)")
print()
print("4. For VCF data specifically:")
print("   result = process_vcf_with_alt_splitting(your_vcf_lazy_frame)")

# Real-world usage pattern:
usage_code = '''
# Load VCF file
vcf_data = gb.read_vcf_file("your_file.vcf")

# Split ALT alleles into separate rows
split_data = flatmap_alt_alleles_flexible(vcf_data)

# Collect results
result = split_data.collect()
'''

print(f"\nReal-world usage pattern:\n{usage_code}")
